In [1]:
### Opening and parsing URL
from lxml.html import parse 
from urllib2 import urlopen
parsed = parse(urlopen('http://finance.yahoo.com/q/op?s=AAPL+Options'))
doc = parsed.getroot()

In [2]:
### Extracting links tags
links = doc.findall('.//a')
links[10:15]

[<Element a at 0x854dd18>,
 <Element a at 0x854dd68>,
 <Element a at 0x854ddb8>,
 <Element a at 0x854de08>,
 <Element a at 0x854de58>]

In [3]:
### Changing HTML elements to text
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls[10:15]

['https://mobile.yahoo.com/',
 'http://everything.yahoo.com/',
 'https://www.yahoo.com/politics',
 'https://www.yahoo.com/movies',
 'https://www.yahoo.com/music']

In [4]:
### Extracting table tags and assigning example to the first table
tables = doc.findall('.//table') 
example=tables[2]

In [5]:
### Extracting all rows of example table
rows = example.findall('.//tr')
### Extracting all elements of rows including headers row
def _unpack(row, kind='td'): ### th kind refers to header row and td refers to other rows   
    elts = row.findall('.//%s' % kind)    
    return [val.text_content() for val in elts]

In [6]:
### Unpack header row
print (_unpack(rows[0], kind='th'))

[u'\n                \n                     Strike\n                    \n                        \ue004\n                        \ue002\n                    \n                \n                \u2235 Filter\n            ', 'Contract Name', u'\n                \n                    Last\n                    \n                        \ue004\n                        \ue002\n                    \n                \n            ', u'\n                \n                    Bid\n                    \n                        \ue004\n                        \ue002\n                    \n                \n            ', u'\n                \n                    Ask\n                    \n                        \ue004\n                        \ue002\n                    \n                \n            ', u'\n                \n                    Change\n                    \n                        \ue004\n                        \ue002\n                    \n                \n            ', u'\

In [7]:
### Unpack fifth row
print (_unpack(rows[5], kind='td'))

['\n            78.00\n        ', '\n            AAPL160603P00078000\n        ', '\n            0.01\n        ', '\n            0.00\n        ', '\n            0.02\n        ', '\n            0.00\n        ', '\n            \n                0.00%\n            \n        ', '\n            2\n        ', '\n            31\n        ', '\n            68.75%\n        ']


In [8]:
### Parsing all elements of the tables including header rows
from pandas.io.parsers import TextParser
def parse_options_data(table):
    rows = table.findall('.//tr')
    header = _unpack(rows[0], kind='th')    
    data = [_unpack(r) for r in rows[1:]]    
    return TextParser(data, names=header).get_chunk()

In [10]:
example_data = parse_options_data(example)
example_data[1:3]

,Strike   ∵ Filter,Contract Name,Last  ,Bid  ,Ask  ,Change  ,%Change  ,Volume  ,Open Interest  ,Implied Volatility  
1,\n 75.00\n,\n AAPL160603P00075000\n,0.01,0.0,0.01,0.0,\n \n 0.00%\n ...,1.0,185.0,\n 71.88%\n
2,\n 76.00\n,\n AAPL160603P00076000\n,0.07,0.0,0.01,0.0,\n \n 0.00%\n ...,1.0,11.0,\n 68.75%\n
